In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:

train = pd.read_csv("train_new.csv")
test = pd.read_csv("test_new.csv")
idcol = test['id']
#train.shape

In [24]:
#train['comment_text']

In [4]:
#df = pd.concat([train['comment_text'], test['comment_text']], axis=0)

In [6]:
#type(df)

In [3]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_y = train[label_cols]

In [4]:
#df = df.fillna("unknown")
nrow_train = train.shape[0]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2)

In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
#x = list(df)

NameError: name 'df' is not defined

In [9]:
corpus_text = []
for i in range(len(train['comment_text'])):
    c = train['comment_text'][i]
    c= c.lower()
    c = c.split()
   # ps = PorterStemmer()
    c = [stemmer.stem(word) for word in c]
    c = ' '.join(c)
    corpus_text.append(c)

x = corpus_text
#stemmer.stem(x)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer

#vect = TfidfVectorizer(max_features=50000, lowercase=True, analyzer='word',stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
wordvect = TfidfVectorizer( lowercase=True, analyzer='word',stop_words= 'english', token_pattern=u'(?ui)\\b[a-z][a-z][a-z]+\\b',
                       max_features=10000, min_df=2 ,ngram_range=(1,2),dtype=np.float32)

#charvect = TfidfVectorizer( lowercase=True, analyzer='char', max_features=30000,ngram_range=(1,5),token_pattern=u'(?ui)\\b[a-z][a-z][a-z]+\\b')

In [13]:
wordfeatures = wordvect.fit(x)
#charfeatures = charvect.fit(x)

In [14]:
train_word_features = wordvect.transform(train['comment_text'])
test_word_features = wordvect.transform(test['comment_text'])

#turain_char_features = charvect.transform(train['comment_text'])
#test_char_features = charvect.transform(test['comment_text'])

In [15]:
train =  train_word_features
test = test_word_features

In [18]:
test.shape

(153164, 10000)

In [17]:
from sklearn.linear_model import LogisticRegression

In [20]:
pred = np.zeros((test.shape[0], len(label_cols)))

cv_score =[]
for i,col in enumerate(label_cols):
    model = LogisticRegression(C=10,random_state = i)
    print('Building {} model for column:{''}'.format(i,col)) 
    model.fit(train,train_y[col])
    #cv_score.append(lr.score)
    pred[:,i] = model.predict_proba(test)[:,1]
    print("Accuracy on training set: {:.3f}".format(model.score(train, train_y[col])))

Building 0 model for column:toxic
Accuracy on training set: 0.962
Building 1 model for column:severe_toxic
Accuracy on training set: 0.992
Building 2 model for column:obscene
Accuracy on training set: 0.980
Building 3 model for column:threat
Accuracy on training set: 0.998
Building 4 model for column:insult
Accuracy on training set: 0.975
Building 5 model for column:identity_hate
Accuracy on training set: 0.994


In [137]:
#prd_1 = pd.DataFrame(pred,columns=label_cols)
#submit = pd.concat([idcol,prd_1],axis=1)
#submit.to_csv('toxic_lr.csv.gz',compression='gzip',index=False)
#submit.to_csv('lR_train_tesy.csv',index=False)
#submit.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999944,0.108471,0.999901,0.025119,0.947308,0.380705
1,0000247867823ef7,0.001244,0.000478,0.000395,0.000396,0.002501,0.000590
2,00013b17ad220c46,0.016559,0.000705,0.004303,0.000198,0.006815,0.001579
3,00017563c3f7919a,0.000897,0.001476,0.001018,0.000132,0.002144,0.000049
4,00017695ad8997eb,0.004431,0.001013,0.001309,0.000838,0.002365,0.000629
